In [ ]:
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/My Drive/Colab Notebooks/SDC/driving_dataset.zip", 'r')
zip_ref.extractall("/content/data")
zip_ref.close()

In [ ]:
import pandas
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing import image
import keras

data = pandas.read_csv('/content/drive/My Drive/Colab Notebooks/SDC/data.txt',header = None,sep=' ' )

In [ ]:
data=data.values

In [ ]:
x_data = data[:,0]

In [ ]:
y_data=data[:,-1]

In [ ]:
length=int((x_data.shape[0])*0.8)
xtrain = x_data[:length]
ytrain = y_data[:length]
xtest = x_data[length:]
ytest = y_data[length:]

In [ ]:
import random
c = list(zip(xtrain,ytrain))
random.shuffle(c)
xtrain,ytrain = zip(*c)


c = list(zip(xtest,ytest))
random.shuffle(c)
xtest,ytest = zip(*c)

In [ ]:
xtrain[200]

'35763.jpg'

In [ ]:
from keras.models import Sequential
from keras.layers import Cropping2D,Conv2D,Flatten, Dense, Dropout, BatchNormalization, LeakyReLU, MaxPool2D
from keras.optimizers import Adam
from keras import regularizers

In [ ]:
import tensorflow as tf
from keras.layers import Lambda

In [ ]:

model = Sequential()


model.add(Cropping2D(((80,0), (0, 0)), input_shape=(160, 320, 3)))
model.add(Lambda(lambda x: tf.image.resize(x, (66, 200))))

model.add(Conv2D(24, (5, 5), activation='relu', input_shape=(200, 200, 3), strides=(2, 2), kernel_regularizer=regularizers.l2(0.001)))

model.add(BatchNormalization())

model.add(Conv2D(36, (5, 5), activation='relu', strides=(2, 2), kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())

model.add(Conv2D(48, (5, 5), activation='relu', strides=(2, 2), kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3), activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())

model.add(Conv2D(128, (3, 3), activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(1164, activation='relu',kernel_regularizer=regularizers.l2(0.001)))

model.add(Dense(200, activation='relu',kernel_regularizer=regularizers.l2(0.001)))

model.add(Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.001)))

model.add(Dense(50, activation='relu',kernel_regularizer=regularizers.l2(0.001)))

model.add(Dense(1, activation='tanh', kernel_regularizer=regularizers.l2(0.001)))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cropping2d_1 (Cropping2D)    (None, 80, 320, 3)        0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 66, 200, 3)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 31, 98, 24)        1824      
_________________________________________________________________
batch_normalization_5 (Batch (None, 31, 98, 24)        96        
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 47, 36)        21636     
_________________________________________________________________
batch_normalization_6 (Batch (None, 14, 47, 36)        144       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 5, 22, 48)        

In [ ]:
model.compile(optimizer = Adam(0.001),loss = 'mse',metrics=['mae'])

In [ ]:
import cv2

In [ ]:
from keras.preprocessing import image

def LoadBatch(batch_size,i,xdata,ydata):
  xd=[]
  yd=[]
  pointer=0
  if i==len(xdata)//batch_size-1:
    batch_size=len(xdata)%batch_size
  while pointer<batch_size:
    img=image.load_img("/content/data/driving_dataset/"+xdata[(pointer+i*batch_size)%l],color_mode='rgb',target_size=[160,320])
    img = image.img_to_array(img)/255.0
    xd.append(img)
    yd.append(ydata[(pointer+i*batch_size)%l]*np.pi/180)
    pointer = pointer+1
  return np.array(xd),np.array(yd)

In [ ]:
c=0
l=len(xtest)
epochs=5
batch_size=128
num=len(xtrain)
for epoch in range(epochs):
    for i in range(int(num//batch_size)):
        xs, ys = LoadBatch(batch_size,i,xtrain,ytrain)
        ys.reshape(-1,1)
        model.fit(xs,ys,verbose=0)

        if i%10 == 0:
            score_train = model.evaluate(xs,ys,verbose=0)
            xs1, ys1 = LoadBatch(batch_size,c,xtest,ytest)
            c+=1
            ys1.reshape(-1,1)
            score_test = model.evaluate(xs1,ys1,verbose=0)
            print("Epoch:%d Step:%d Train loss:%f Train mae:%f Test loss:%f Test mae:%f\n"%(epoch+1,i,score_train[0],score_train[1],score_test[0],score_test[1]))
        if i%100==0:
           model.save_weights("/content/drive/My Drive/weightsfinal.h5")

Epoch:1 Step:0 Train loss:2.538924 Train mae:0.262939 Test loss:2.434596 Test mae:0.198002

Epoch:1 Step:10 Train loss:2.484872 Train mae:0.290511 Test loss:2.527164 Test mae:0.267014

Epoch:1 Step:20 Train loss:2.264842 Train mae:0.302864 Test loss:2.290179 Test mae:0.298919

Epoch:1 Step:30 Train loss:2.073703 Train mae:0.363122 Test loss:1.894771 Test mae:0.298780

Epoch:1 Step:40 Train loss:1.719134 Train mae:0.307938 Test loss:1.852037 Test mae:0.384802

Epoch:1 Step:50 Train loss:1.462034 Train mae:0.247580 Test loss:1.447530 Test mae:0.194520

Epoch:1 Step:60 Train loss:1.382548 Train mae:0.322793 Test loss:1.374821 Test mae:0.331787

Epoch:1 Step:70 Train loss:1.158234 Train mae:0.260902 Test loss:1.580303 Test mae:0.582201

Epoch:1 Step:80 Train loss:0.988323 Train mae:0.258602 Test loss:1.059455 Test mae:0.276478

Epoch:1 Step:90 Train loss:0.820432 Train mae:0.206066 Test loss:0.994776 Test mae:0.359906

Epoch:1 Step:100 Train loss:1.017567 Train mae:0.316066 Test loss:0.908